In [73]:
import pandas as pd
from pathlib import Path

from sklearn.pipeline import make_pipeline
from yellowbrick.model_selection import LearningCurve
from yellowbrick.regressor import ResidualsPlot
from yellowbrick.regressor import PredictionError

from sklearn.gaussian_process.kernels import RBF,WhiteKernel,ConstantKernel
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.feature_selection import VarianceThreshold
from sklearn.model_selection import KFold

from imblearn import over_sampling as ovs
from sklearn.decomposition import PCA

import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.preprocessing import StandardScaler, RobustScaler, MinMaxScaler
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error,mean_absolute_error,r2_score
from sklearn import metrics
from sklearn.externals import joblib
from sklearn.model_selection import GridSearchCV,RepeatedKFold
from sklearn.model_selection import train_test_split

%matplotlib inline

In [74]:
ConstantKernel?

In [2]:
sns.set_context(context="paper")


In [6]:
ABPRED_DIR = Path().cwd().parent

#dataframe final
df_final = pd.read_csv("../data/DF_contact400_energy_2019.f_corr.csv",index_col=0)
# Quitar modelos por homologia deltraining set
#df_final_onlyHM = df_final.loc[df_final.index.str.startswith("HM")]
#df_final= df_final.loc[~df_final.index.str.startswith("HM")]

index_ddg8 = (df_final['ddG_exp']==8)
df_final = df_final.loc[-index_ddg8]
#testiar eliminando estructuras con ddg menor o igual a -4 kcal/mol , outliers
index_ddg_4 =  (df_final['ddG_exp'] <= -4)
df_final = df_final.loc[-index_ddg_4]


pdb_names = df_final.index
features_names = df_final.drop('ddG_exp',axis=1).columns


In [7]:
X = df_final.drop('ddG_exp',axis=1)
y = df_final['ddG_exp']

In [8]:
# split for final test
X_train, X_test, y_train, y_test = train_test_split(X, y,train_size=0.8,random_state=13)


/Users/vfica/miniconda3/envs/bio/lib/python3.6/site-packages/sklearn/model_selection/_split.py:2069: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


In [24]:
cross_val_score?

In [78]:
# Load a regression dataset

selector = VarianceThreshold()
scaler = MinMaxScaler()
#scaler = StandardScaler()
#2)
kernel = 0.5 * RBF(length_scale=0.5, length_scale_bounds=(1e-2, 1e3)) + WhiteKernel(noise_level=1, noise_level_bounds=(1e-10, 1e+1)) + ConstantKernel()
    
    
gr_model = GaussianProcessRegressor(random_state=1313,kernel=kernel,normalize_y=True)
#3) Crear pipeline
pipeline1 = make_pipeline(scaler,selector,gr_model)

cv = RepeatedKFold(n_splits=10,n_repeats=5,random_state=13)
# 5) hiperparametros a ajustar
r2 = cross_val_score(X=X_train,y=y_train,estimator=pipeline1,n_jobs=-1,cv=cv,verbose=5)
print(r2.mean(),r2.std())

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done  30 out of  50 | elapsed:   11.1s remaining:    7.4s
[Parallel(n_jobs=-1)]: Done  41 out of  50 | elapsed:   14.5s remaining:    3.2s


0.21957338767309806 0.21334239279764433


[Parallel(n_jobs=-1)]: Done  50 out of  50 | elapsed:   16.5s finished


In [63]:
r2

array([ 0.24969224,  0.34808461,  0.49511477,  0.30057004,  0.39257468,
       -0.19888877, -0.00784752,  0.2390412 ,  0.38914283,  0.0142972 ,
        0.16544804,  0.17277347,  0.38672799,  0.46884781,  0.01699609,
        0.33180147, -0.04166567,  0.52426623,  0.29227418,  0.078493  ,
        0.14472702,  0.400538  , -0.01037986,  0.29931374, -0.18815049,
        0.17493999,  0.32506785,  0.40520115,  0.41974362,  0.38395343,
        0.1913373 ,  0.25841777,  0.34932472,  0.38571116,  0.40057936,
       -0.55929095,  0.37501876,  0.37055611, -0.16660863,  0.34932892,
        0.19085178,  0.28652937,  0.22922829,  0.43184138,  0.21809338,
        0.2342773 ,  0.41722203,  0.19886964, -0.07335364,  0.04824408])

In [12]:
# index of best scores
rmse_bestCV_test_index = grid1.cv_results_['mean_test_neg_mean_squared_error'].argmax()
rmse_bestCV_train_index = grid1.cv_results_['mean_train_neg_mean_squared_error'].argmax()
r2_bestCV_test_index = grid1.cv_results_['mean_test_r2'].argmax()
r2_bestCV_train_index = grid1.cv_results_['mean_train_r2'].argmax()
# scores
rmse_bestCV_test_score = grid1.cv_results_['mean_test_neg_mean_squared_error'][rmse_bestCV_test_index]
rmse_bestCV_test_std = grid1.cv_results_['std_test_neg_mean_squared_error'][rmse_bestCV_test_index]
rmse_bestCV_train_score = grid1.cv_results_['mean_train_neg_mean_squared_error'][rmse_bestCV_train_index]
rmse_bestCV_train_std = grid1.cv_results_['std_train_neg_mean_squared_error'][rmse_bestCV_train_index]

r2_bestCV_test_score = grid1.cv_results_['mean_test_r2'][r2_bestCV_test_index]
r2_bestCV_test_std = grid1.cv_results_['std_test_r2'][r2_bestCV_test_index]
r2_bestCV_train_score = grid1.cv_results_['mean_train_r2'][r2_bestCV_train_index]
r2_bestCV_train_std = grid1.cv_results_['std_train_r2'][r2_bestCV_train_index]

print('CV test RMSE {:f} +/- {:f}'.format(np.sqrt(-rmse_bestCV_test_score),np.sqrt(rmse_bestCV_test_std)))
print('CV train RMSE {:f} +/- {:f}'.format(np.sqrt(-rmse_bestCV_train_score),np.sqrt(rmse_bestCV_train_std)))
print('CV test r2 {:f} +/- {:f}'.format(r2_bestCV_test_score,r2_bestCV_test_std))
print('CV train r2 {:f} +/- {:f}'.format(r2_bestCV_train_score,r2_bestCV_train_std))

print(grid1.best_params_)
y_test_pred = grid1.best_estimator_.predict(X_test)
y_train_pred = grid1.best_estimator_.predict(X_train)
print("\nRMSE for test dataset: {}".format(np.round(np.sqrt(mean_squared_error(y_test, y_test_pred)), 2)))
print("RMSE for train dataset: {}".format(np.round(np.sqrt(mean_squared_error(y_train, y_train_pred)), 2)))
print("pearson corr  {:f}".format(np.corrcoef(y_test_pred,y_test)[0][1]))
print('R2 test',grid1.score(X_test,y_test))
print('R2 train',grid1.score(X_train,y_train))


CV test RMSE 10.112911 +/- 9.887791
CV train RMSE 0.029923 +/- 0.019206
CV test r2 -51.974290 +/- 51.329487
CV train r2 0.999573 +/- 0.000178
{'gaussianprocessregressor__n_restarts_optimizer': 1}

RMSE for test dataset: 15.43
RMSE for train dataset: 0.03
pearson corr  -0.064155
R2 test -116.53819209167602
R2 train 0.9995261843324637
